## **Binary-Classification**

In [3]:
!pip install -q dotenv

In [4]:
import sys
import os
sys.path.append(os.path.abspath('../'))

In [7]:
from src import load_data, InsomniaClassifier, convert_output_to_json

val_path = "/home/ubuntu/SSM/validation/combined_insomnia_data_validation.csv"
df = load_data(val_path)
clinical_notes = df['text'].tolist()
classifier = InsomniaClassifier()

classification_results = []
extracted_texts = []

Successfully loaded 20 records from /home/ubuntu/SSM/validation/combined_insomnia_data_validation.csv


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda


In [8]:
df.head()

,note_id,text,Definition 1,Definition 2,Rule A,Rule B,Rule C,Insomnia
0,1280230,female patient in sixties prescribed no drugs\...,yes,no,no,no,no,no
1,1286616,male patient in fifties prescribed no drugs\n\...,yes,yes,yes,no,yes,yes
2,1291281,female patient in seventies prescribed no drug...,no,no,no,no,no,no
3,1293851,female patient in seventies prescribed Nitrogl...,yes,yes,yes,yes,yes,yes
4,1295870,male patient in seventies prescribed Pantopraz...,yes,yes,yes,no,yes,yes


In [9]:
# Process each clinical note with exception handling
for idx, clinical_note in enumerate(clinical_notes):
    print(f"Processing text {idx + 1}/{len(clinical_notes)}: {clinical_note[:100]}...")
    try:
        classification, extracted = classifier.classify(clinical_note)
        classification_results.append(classification)
        extracted_texts.append(extracted)
    except RuntimeError as e:
        print(f"RuntimeError for text at index {idx}: {e}")
        # Append default classifications and empty extracted text on error
        classification_results.append({
            "Definition 1 (Sleep Difficulty)": "no",
            "Definition 2 (Daytime Impairment)": "no",
            "Rule A (Insomnia Diagnosis)": "no",
            "Rule B (Primary Medications)": "no",
            "Rule C (Secondary Medications)": "no",
            "Final Insomnia Status": "no"
        })
        extracted_texts.append({
            "Definition 1 Extracted": "",
            "Definition 2 Extracted": "",
            "Rule A Extracted": "",
            "Rule B Extracted": "",
            "Rule C Extracted": ""
        })
    print("-" * 80)

Processing text 1/20: female patient in sixties prescribed no drugs

NPN 7p-7a:
     RESP: pt cont with shallow breathing ...


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model Response:
 ### ✅ Response Format:
Sleep Difficulty Phrases:
unable to sleep on either bipap
waking up to sternal rub, coughed up mod amts yellow secretions
 
Daytime Impairment Phrases:
can't concentrate 
complains about sleep quality
########################################
unable to sleep on either bipap
waking up to sternal rub, coughed up mod amts yellow secretions
can't concentrate 
complains about sleep quality
--------------------------------------------------------------------------------
Processing text 2/20: male patient in fifties prescribed no drugs

MICU NPN 7P-7A
Neuro: Pt. AAO x 3, odd affect. A little...
Model Response:
 ### ✅ Response Format:
Sleep Difficulty Phrases:
[unknown]

Daytime Impairment Phrases:
[Pt. A little confused/forgetful re: events of the past few days. MAE. Received 10 mg Valium @ 0300 for insomnia with some effect.]
########################################
[unknown]
[Pt. A little confused/forgetful re: events of the past few days. MAE. Receive

In [12]:
import pandas as pd
# Convert results to DataFrames
df_classification = pd.DataFrame(classification_results)
df_extracted = pd.DataFrame(extracted_texts)

# Combine all DataFrames
df_final = pd.concat([df[['text', 'note_id']], df_classification, df_extracted], axis=1)

# Rename columns to match expected names for JSON conversion
df_final = df_final.rename(columns={
    "Definition 1 (Sleep Difficulty)": "Definition 1 Pred",
    "Definition 2 (Daytime Impairment)": "Definition 2 Pred",
    "Rule A (Insomnia Diagnosis)": "Rule A Pred",
    "Rule B (Primary Medications)": "Rule B Pred",
    "Rule C (Secondary Medications)": "Rule C Pred",
    "Final Insomnia Status": "Insomnia Pred",
    "Definition 1 Extracted": "Definition 1 Evidence",
    "Definition 2 Extracted": "Definition 2 Evidence",
    "Rule B Extracted": "Rule B Evidence",
    "Rule C Extracted": "Rule C Evidence"
})

In [21]:
output_csv_dir = "/home/ubuntu/Downloads/"
#os.makedirs(output_csv_dir, exist_ok=True)
# Save CSV to results folder
csv_output_path = os.path.join(output_csv_dir, "output.csv")
df_final.to_csv(csv_output_path, index=False)

In [22]:
# Generate JSON outputs
convert_output_to_json(csv_output_path)

JSON file saved at results/subtask_1.json
JSON file saved at results/subtask_2a.json
JSON file saved at results/subtask_2b.json
